# PaLM REST API: Tuning Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://developers.generativeai.google/tutorials/tuning_quickstart_rest"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://developers.generativeai.google/tutorials/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this notebook, you'll learn how to get started with the PaLM API tuning service using curl commands or the Python request API to call the PaLM REST API. Here, you'll learn how to tune the text model behind the PaLM API's text generation service.

**Note**: At this time, the PaLM API is [only available in certain regions](https://developers.generativeai.google/available_regions).

## Setup

### Authenticate

The PaLM API lets you tune models on your own data. Since it's your data and
your tuned models this needs stricter access controls than API-Keys can provide.

Before you can run this tutorial, you'll need to
[setup OAuth for your project](oauth_quickstart.ipynb).

If you want to run this notebook in Colab start by uploading your
`client_secret*.json` file using the "File > Upload" option.

![Show colab's File > Upload option](https://developers.generativeai.google/tutorials/images/colab_upload.png)

In [ ]:
!cp client_secret*.json client_secret.json
!ls

client_secret.json

This gcloud command turns the `client_secret.json` file into credentials that can be used to authenticate with the service.

Important: If you're running this in Colab, **don't just click the link it prints**. That will fail. Follow the instriuctions and copy the `gcloud` command it prints to your local machine and run it there, then paste the output from your local machine back here.

In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

## Calling the REST API with CURL

This section gives example curl statements to call the REST API. You will learn how to create a tuning job, check its status and once complete, make an inference call.

### Set variables

Set variables for recurring values to use for the rest of the REST API calls. The code is using the Python `os` library to set environment variables which is accessible in all the code cells.

This is specific to the Colab notebook environment. The code in the next code cell is equivalent to running the following commands in a bash terminal.

```bash
export access_token=$(gcloud auth application-default print-access-token)
export project_id=my-project-id
export base_url=https://generativelanguage.googleapis.com
```

In [ ]:
import os

access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

os.environ['access_token'] = access_token
os.environ['project_id'] = "project-id"
os.environ['base_url'] = "https://generativelanguage.googleapis.com"


### List tuned models

Verify your authentication setup by listing the currently available tuned models.

In [ ]:
%%bash

curl -X GET ${base_url}/v1beta3/tunedModels \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" | grep name


      "name": "tunedModels/testnumbergenerator-fvitocr834l6",
      "name": "tunedModels/my-display-name-81-9wpmc1m920vq",
      "displayName": "my display name 81",
      "name": "tunedModels/number-generator-model-kctlevca1g3q",
      "name": "tunedModels/my-display-name-81-r9wcuda14lyy",
      "displayName": "my display name 81",
      "name": "tunedModels/number-generator-model-w1eabln5adwp",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17583    0 17583    0     0  51600      0 --:--:-- --:--:-- --:--:-- 51563


### Create tuned model

To create a tuned model, you need to pass your dataset to the model in the `training_data` field.

For this example, you will tune a model to generate the next number in the sequence. For example, if the input is `1`, the model should output `2`. If the input is `one hundred`, the output should be `one hundred one`.

In [ ]:
%%bash

curl -X POST ${base_url}/v1beta3/tunedModels \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \
    -d '
      {
        "display_name": "number generator model",
        "base_model": "models/text-bison-001",
        "tuning_task": {
          "hyperparameters": {
            "batch_size": 2,
            "learning_rate": 0.001,
            "epoch_count":3,
          },
          "training_data": {
            "examples": {
              "examples": [
                {
                    "text_input": "1",
                    "output": "2",
                },{
                    "text_input": "3",
                    "output": "4",
                },{
                    "text_input": "-3",
                    "output": "-2",
                },{
                    "text_input": "twenty two",
                    "output": "twenty three",
                },{
                    "text_input": "two hundred",
                    "output": "two hundred one",
                },{
                    "text_input": "ninety nine",
                    "output": "one hundred",
                },{
                    "text_input": "8",
                    "output": "9",
                },{
                    "text_input": "-98",
                    "output": "-97",
                },{
                    "text_input": "1,000",
                    "output": "1,001",
                },{
                    "text_input": "10,100,000",
                    "output": "10,100,001",
                },{
                    "text_input": "thirteen",
                    "output": "fourteen",
                },{
                    "text_input": "eighty",
                    "output": "eighty one",
                },{
                    "text_input": "one",
                    "output": "two",
                },{
                    "text_input": "three",
                    "output": "four",
                },{
                    "text_input": "seven",
                    "output": "eight",
                }
              ]
            }
          }
        }
      }' | tee tunemodel.json


{
  "name": "tunedModels/number-generator-model-q2d0uism5ivd/operations/xvyx09sjxlmh",
  "metadata": {
    "@type": "type.googleapis.com/google.ai.generativelanguage.v1beta3.CreateTunedModelMetadata",
    "totalSteps": 23,
    "tunedModel": "tunedModels/number-generator-model-q2d0uism5ivd"
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2277    0   297  100  1980    146    975  0:00:02  0:00:02 --:--:--  1121


### Get tuned model state

The state of the model is set to `CREATING` during training and will change to `ACTIVE` once its complete.

Below is a bit of python code to parse out the generated model name from the response JSON. If you're running this in a terminal you can try using a bash JSON parser to parse the response.

In [ ]:
import json

first_page = json.load(open('tunemodel.json'))
os.environ['modelname'] = first_page['metadata']['tunedModel']

print(os.environ['modelname'])


tunedModels/number-generator-model-q2d0uism5ivd


  Do another `GET` request with the model name to get the model metadata which includes the state field.

In [ ]:
%%bash

curl -X GET ${base_url}/v1beta3/${modelname} \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \ | grep state


  "state": "CREATING",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   494    0   494    0     0    760      0 --:--:-- --:--:-- --:--:--   760
curl: (3) URL using bad/illegal format or missing URL


### Run inference

Once your tuning job is finished, you can use it to generate text with the text service.


In [ ]:
%%bash

curl -X POST ${base_url}/v1beta3/${modelname}:generateText \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \
    -d '{
        "prompt": {
              "text": "4"
              },
        "temperature": 1.0,
        "candidate_count": 2}' | grep output

      "output": "3 2 1",
      "output": "3 2",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1569    0  1447  100   122    183     15  0:00:08  0:00:07  0:00:01   310


The output from your model may or may not be correct. If the tuned model isn't performing up to your required standards, you can try adding more high quality examples, tweaking the hyperparameters or adding a preamble to your examples. You can even create another tuned model based on the first one you created.

See the [tuning guide](../guide/model_tuning_guidance) for more guidance on improving performance.

## Call the REST API with Python requests

You can call the rest API with any library that allows you to send http requests.
The next set of examples use the Python requests library, and demonstrates some of the  more advanced features.

### Set variables

In [ ]:
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

project = 'project-id'
base_url = "https://generativelanguage.googleapis.com"


Import the `requests` library.

In [ ]:
import requests
import json

### List tuned models

Verify your authentication setup by listing the currently available tuned models.

In [ ]:
headers={
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'x-goog-user-project': project
}

result = requests.get(
  url=f'{base_url}/v1beta3/tunedModels',
  headers = headers,
)

In [ ]:
result.json()

{'tunedModels': [{'name': 'tunedModels/testnumbergenerator-fvitocr834l6',
   'baseModel': 'models/text-bison-001',
   'displayName': 'test_number_generator',
   'description': '{"description":"generates the  next number in the sequence given the input text","exampleInput":"input: 1","exampleOutput":"output: 2","datasourceUrl":"https://drive.google.com/open?id=11Pdm6GNom4vlBMUHwO6yFjGQT3t1yi44WVShXMFnkVA&authuser=0&resourcekey=0-2d17tccbdBoThXMkNDvtag","showedTuningComplete":false}',
   'state': 'ACTIVE',
   'createTime': '2023-09-18T11:06:39.092786Z',
   'updateTime': '2023-09-18T11:07:24.198359Z',
   'tuningTask': {'startTime': '2023-09-18T11:06:39.461814784Z',
    'completeTime': '2023-09-18T11:07:24.198359Z',
    'snapshots': [{'step': 1,
      'meanLoss': 16.613504,
      'computeTime': '2023-09-18T11:06:44.532937624Z'},
     {'step': 2,
      'epoch': 1,
      'meanLoss': 20.299532,
      'computeTime': '2023-09-18T11:06:47.825134421Z'},
     {'step': 3,
      'epoch': 1,
      'm

### Create tuned model

Same as for the Curl example, you pass in the dataset through the `training_data` field.

In [ ]:
operation = requests.post(
    url = f'{base_url}/v1beta3/tunedModels',
    headers=headers,
    json= {
        "display_name": "number generator",
        "base_model": "models/text-bison-001",
        "tuning_task": {
          "hyperparameters": {
            "batch_size": 4,
            "learning_rate": 0.001,
            "epoch_count":3,
          },
          "training_data": {
            "examples": {
              "examples": [
                {
                    'text_input': '1',
                    'output': '2',
                },{
                    'text_input': '3',
                    'output': '4',
                },{
                    'text_input': '-3',
                    'output': '-2',
                },{
                    'text_input': 'twenty two',
                    'output': 'twenty three',
                },{
                    'text_input': 'two hundred',
                    'output': 'two hundred one',
                },{
                    'text_input': 'ninety nine',
                    'output': 'one hundred',
                },{
                    'text_input': '8',
                    'output': '9',
                },{
                    'text_input': '-98',
                    'output': '-97',
                },{
                    'text_input': '1,000',
                    'output': '1,001',
                },{
                    'text_input': '10,100,000',
                    'output': '10,100,001',
                },{
                    'text_input': 'thirteen',
                    'output': 'fourteen',
                },{
                    'text_input': 'eighty',
                    'output': 'eighty one',
                },{
                    'text_input': 'one',
                    'output': 'two',
                },{
                    'text_input': 'three',
                    'output': 'four',
                },{
                    'text_input': 'seven',
                    'output': 'eight',
                }
              ]
            }
          }
        }
      }
)

In [ ]:
operation

<Response [200]>

In [ ]:
operation.json()

{'name': 'tunedModels/number-generator-ncqqnysl74dt/operations/qqlbwzfyzn0k',
 'metadata': {'@type': 'type.googleapis.com/google.ai.generativelanguage.v1beta3.CreateTunedModelMetadata',
  'totalSteps': 12,
  'tunedModel': 'tunedModels/number-generator-ncqqnysl74dt'}}

Set a variable with the name of your tuned model to use for the rest of the calls.

In [ ]:
name=operation.json()["metadata"]["tunedModel"]
name


'tunedModels/number-generator-ncqqnysl74dt'

### Get tuned model state

You can check the progress of your tuning job by checking the state field. `CREATING` means the tuning job is still ongoing and `ACTIVE` means the trainins is complete and the tuned model is ready to use.

In [ ]:
tuned_model = requests.get(
    url = f'{base_url}/v1beta3/{name}',
    headers=headers,
)

In [ ]:
tuned_model.json()

{'name': 'tunedModels/number-generator-ncqqnysl74dt',
 'baseModel': 'models/text-bison-001',
 'displayName': 'number generator',
 'state': 'CREATING',
 'createTime': '2023-09-19T19:56:25.999303Z',
 'updateTime': '2023-09-19T19:56:25.999303Z',
 'tuningTask': {'startTime': '2023-09-19T19:56:26.297862545Z',
  'hyperparameters': {'epochCount': 3, 'batchSize': 4, 'learningRate': 0.001}},
 'temperature': 0.7,
 'topP': 0.95,
 'topK': 40}

The code below checks the state field every 5 seconds until it is no longer in the `CREATING` state.

In [ ]:
import time
import pprint

op_json = operation.json()
response = op_json.get('response')
error = op_json.get('error')

while response is None and error is None:
    time.sleep(31)

    operation = requests.get(
        url = f'{base_url}/v1/{op_json["name"]}',
        headers=headers,
    )

    op_json = operation.json()
    response = op_json.get('response')
    error = op_json.get('error')

    percent = op_json['metadata'].get('completedPercent')
    if percent is not None:
      print(f"{percent:.2f}% - {op_json['metadata']['snapshots'][-1]}")
      print()

if error is not None:
    raise Exception(error)

21.28% - {'step': 40, 'epoch': 10, 'meanLoss': 2.4871845, 'computeTime': '2023-09-20T00:23:55.255785843Z'}

21.28% - {'step': 40, 'epoch': 10, 'meanLoss': 2.4871845, 'computeTime': '2023-09-20T00:23:55.255785843Z'}

43.09% - {'step': 81, 'epoch': 21, 'meanLoss': 0.032220088, 'computeTime': '2023-09-20T00:24:56.302837803Z'}

43.09% - {'step': 81, 'epoch': 21, 'meanLoss': 0.032220088, 'computeTime': '2023-09-20T00:24:56.302837803Z'}

63.83% - {'step': 120, 'epoch': 32, 'meanLoss': 0.0030430648, 'computeTime': '2023-09-20T00:25:57.228615435Z'}

63.83% - {'step': 120, 'epoch': 32, 'meanLoss': 0.0030430648, 'computeTime': '2023-09-20T00:25:57.228615435Z'}

85.11% - {'step': 160, 'epoch': 42, 'meanLoss': -1.1145603e-06, 'computeTime': '2023-09-20T00:26:57.819011896Z'}

100.00% - {'step': 188, 'epoch': 50, 'meanLoss': 0.00040101097, 'computeTime': '2023-09-20T00:27:40.024132813Z'}



### Run inference

Once the tuning job is finished, you can use it to generate text in the same way you would use the base text model.

In [ ]:
import time

m = requests.post(
    url = f'{base_url}/v1beta3/{name}:generateText',
    headers=headers,
    json= {
         "prompt": {
              "text": "9"
              },
    })

In [ ]:
import pprint
print(m.json()['candidates'][0]['output'])

9


The output from your model may or may not be correct. If the tuned model isn't performing up to your required standards, you can try adding more high quality examples, tweaking the hyperparameters or adding a preamble to your examples.


## Next steps

* See the [tuning quickstart with Python](tuning_quickstart_python) to start coding with the tuning service.
* See the [tuning guidance](../guide/model_tuning_guidance) for more details on how to best tune the model for your use case.